<a href="https://colab.research.google.com/github/ganesh-codes404/DL-Assignment-2/blob/main/Lyrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers datasets accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
import os
import pandas as pd
poetry_dir = '/content/drive/My Drive/archive-2'
texts = []
for filename in os.listdir(poetry_dir):
    if filename.endswith('.txt'):
        with open(os.path.join(poetry_dir, filename), 'r', encoding='utf-8') as f:
            text = f.read().strip()
            if len(text) > 50:
                texts.append(text)
df = pd.DataFrame(texts, columns=["text"])
print(f"Loaded viable poems", len(df))
df.head()


✅ Loaded 49 poems


,text
0,"Let's stay together I, I'm I'm so in love with..."
1,Can we forget about the things I said when I w...
2,Just before our love got lost you said\nI am a...
3,﻿Eighteen years eighteen years\nShe got one of...
4,I bought my first key from my baby momma broth...


In [4]:
from transformers import GPT2Tokenizer
from datasets import Dataset

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

dataset = Dataset.from_pandas(df)

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=256)
tokenized_dataset = dataset.map(tokenize, batched=True, remove_columns=["text"])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/49 [00:00<?, ? examples/s]

In [5]:
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
import torch

model = GPT2LMHeadModel.from_pretrained("gpt2")

# Training configuration
training_args = TrainingArguments(
    output_dir="./gpt2-lyrics",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=500,
    logging_steps=50,
    fp16=torch.cuda.is_available(),
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Trainer initialization
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)
trainer.train()


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
50,3.265100


TrainOutput(global_step=75, training_loss=3.163380228678385, metrics={'train_runtime': 16.6025, 'train_samples_per_second': 8.854, 'train_steps_per_second': 4.517, 'total_flos': 19204964352000.0, 'train_loss': 3.163380228678385, 'epoch': 3.0})

In [7]:
prompt = "When the sun rises and stars begin to fade"
inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
output = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_length=100,
    do_sample=True,
    top_k=50,
    top_p=0.9,
    temperature=1.0,
    repetition_penalty=1.2,
    num_return_sequences=1,
    pad_token_id=tokenizer.pad_token_id
)

print("Generated Lyrics:\n")
print(tokenizer.decode(output[0], skip_special_tokens=True))


Generated Lyrics:

When the sun rises and stars begin to fade, a song goes:
Be there long before I could sing my songs like an old man singing at his own expense. The day that he wrote 'Nurse of Aime' (in 1940) is over because they were too big for him. He doesn't want us back now; we'll never get out alive anymore! It's not good enough no one wants you in here today if your gonna be rich or low life when everybody else can


In [8]:
from IPython.display import display, Javascript

display(Javascript('IPython.notebook.metadata.widgets = {};'))

<IPython.core.display.Javascript object>